In [12]:
import os
import pybaseball
import pandas as pd
pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', 10)
import numpy as np
import statsapi
from datetime import datetime, timedelta
from dateutil import tz
from geopy.geocoders import Nominatim
import time
import re
import warnings
warnings.filterwarnings('ignore')
from api_keys import visual_crossing
import requests

import warnings
warnings.filterwarnings('ignore')
from pybaseball import statcast, statcast_pitcher, statcast_batter, playerid_lookup
import pandas as pd
pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', 10)
import numpy as np
import statsapi
import pybaseball
from datetime import datetime, timedelta
from dateutil import tz
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, plot_confusion_matrix, precision_score, plot_roc_curve, make_scorer
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.feature_selection import VarianceThreshold

import operator
import math

import xgboost as xgb

import category_encoders as ce

from datetime import datetime, timedelta
from dateutil import tz
from geopy.geocoders import Nominatim

In [13]:
api_counter = 0

In [14]:
def convert_UTC_to_local(row, venue_coords):
    venue_name = row['venue_name']
    game_datetime = row['game_datetime']
    from_zone = tz.gettz('UTC')

    locator = Nominatim(user_agent='myGeocoder')
    try:
        city = venue_coords[venue_coords['NAME'] == row['venue_name']]['CITY'].iloc[0]
        lat = venue_coords[venue_coords['NAME'] == row['venue_name']]['Latitude'].iloc[0]
        lon = venue_coords[venue_coords['NAME'] == row['venue_name']]['Longitude'].iloc[0]
        alt = venue_coords[venue_coords['NAME'] == row['venue_name']]['Altitude'].iloc[0]
    except NameError:
        raise IndexError(venue, index)
    if city == 'Tokyo':
        to_zone = tz.gettz('Asia/Tokyo')
    elif city == 'London':
        to_zone = tz.gettz('Europe/London')
    elif city in ['San Francisco',
                        'Oakland',
                        'Phoenix',
                        'Seattle',
                        'Los Angeles',
                        'San Diego',
                        'Anaheim']:
        to_zone = tz.gettz('America/Los_Angeles')
    elif city == 'Denver':
        to_zone = tz.gettz('America/Denver')
    elif city in ['Minneapolis',
                        'Milwaukee',
                        'Chicago',
                        'St. Louis',
                        'Arlington',
                        'Kansas City',
                        'Houston',
                        'Monterrey',
                        'Omaha',
                        'Dyersville']:
        to_zone = tz.gettz('America/Chicago')
    elif city in ['Buffalo',
                        'Detroit',
                        'Cincinnati',
                        'Pittsburgh',
                        'Tampa Bay',
                        'Philadelphia',
                        'Atlanta',
                        'New York',
                        'Washington',
                        'Cleveland',
                        'Miami',
                        'Boston',
                        'Baltimore',
                        'Toronto',
                        'Williamsport',
                        'Dunedin',
                        'St. Petersburg']:
        to_zone = tz.gettz('America/New_York')
    else:
        raise NameError(venue_name, city)
    utc = datetime.strptime(game_datetime, '%Y-%m-%dT%H:%M:%SZ')
    utc = utc.replace(tzinfo=from_zone)
    local = utc.astimezone(to_zone)

    return(local)

In [15]:
pa_ending_events = np.array(['field_out',
                        'strikeout',
                        'single',
                        'walk',
                        'double',
                        'home_run',
                        'force_out',
                        'grounded_into_double_play',
                        'hit_by_pitch',
                        'field_error',
                        'sac_fly',
                        'triple',
                        'sac_bunt',
                        'fielders_choice',
                        'double_play',
                        'fielders_choice_out',
                        'strikeout_double_play',
                        'catcher_interf',
                        'sac_fly_double_play',
                        'triple_play',
                        'sac_bunt_double_play'])


hit_events = np.array(['single',
            'double',
            'home_run',
            'triple'])

In [16]:
def time_date_weather_data(game_pk_df):
    venue_coords = pd.read_csv('Parks.csv')
    data_collection = {}

    try:
        game_pk_df['coordinates'] = game_pk_df['venue_name'].apply(lambda x: ','.join([str(venue_coords[venue_coords['NAME'] == x]['Latitude'].iloc[0]), str(venue_coords[venue_coords['NAME'] == x]['Longitude'].iloc[0])]))
    except NameError:
        raise IndexError(venue, index)

    game_pk_df = game_pk_df.sort_values(['game_datetime'])

    

    game_pk_df['local_datetime'] = game_pk_df.apply(lambda row: convert_UTC_to_local(row, venue_coords), axis=1)
    game_pk_df['local_datetime'] = game_pk_df['local_datetime'].apply(lambda x: x.strftime("%Y-%m-%dT%H:%M:%S"))
    

    from api_keys import visual_crossing
    import requests

    weather_df = pd.DataFrame()
    for index, row in game_pk_df.iterrows():

#         check if key is past or future
        if datetime.strptime(row['game_datetime'], "%Y-%m-%dT%H:%M:%SZ") > datetime.utcnow():
            URL = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/forecast?locations={row["coordinates"]}&aggregateHours=1&unitGroup=us&forecastDays=1&shortColumnNames=true&contentType=json&key={visual_crossing}'

        else:
            URL = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?&aggregateHours=1&startDateTime={row["local_datetime"]}&endDateTime={row["local_datetime"]}&unitGroup=us&contentType=json&location={row["coordinates"]}&key={visual_crossing}'
        global api_counter
        if api_counter >= 1000:
            print('Prepreprocessing will continue at ' + (datetime.now()+timedelta(seconds = 86400)).strftime('%H:%M:%S'))
            sleep(86400)
            api_counter = 0
        try:
            api_counter += 1
            response = requests.get(URL)
            data = response.json()
            if data['locations'][row['coordinates']]['values'][0]['temp'] == None:
                URL = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/forecast?locations={row["coordinates"]}&aggregateHours=1&unitGroup=us&forecastDays=1&shortColumnNames=true&contentType=json&key={visual_crossing}'
                api_counter += 1
                response = requests.get(URL)
                data = response.json()
                
            if len(data['locations'][row['coordinates']]['values']) > 1:
                for forecast in data['locations'][row['coordinates']]['values']:
                    
                    if forecast['datetimeStr'][:13] == row['local_datetime'][:13]:
                        values = forecast
                        values['datetimeStr'] = row['local_datetime']
                
                
            else:
                values = data['locations'][row['coordinates']]['values'][0]
                values['datetimeStr'] = row['local_datetime']
            data_collection[URL] = data
            weather_df = weather_df.append({**{'coordinates': row['coordinates']}, **values}, ignore_index=True)
    
        except:
            raise IndexError(index, row, response, response.json())
    weather_df = weather_df.rename(columns={'datetimeStr': 'local_datetime'})


    games_and_weather = pd.merge(
        game_pk_df,
        weather_df,
        how="left",
        on=None,
        left_on=['local_datetime', 'coordinates'],
        right_on=['local_datetime', 'coordinates'],
        left_index=False,
        right_index=False,
        sort=False,
        suffixes=("_gpk", "_wdf"),
        copy=True,
        indicator=False,
        validate=None,
    )
    
    return(games_and_weather)

In [17]:
def statcast_importer(year = datetime.now().year):
    if year == datetime.now().year:
        end = (datetime.now()-timedelta(weeks=1)).strftime("%Y-%m-%d")
    else:
        end = str(year) + '-12-31'
    
    start = str(year) + '-01-01'
    df = pybaseball.statcast(str(year) + '-01-01', (datetime.now()-timedelta(weeks=1)).strftime("%Y-%m-%d"))
    df = df[df['game_type'] == 'R']
    print('game types done')
    df['game_date'] = pd.to_datetime(df['game_date'], format='%Y-%m-%d')
    print('datetime done')
    df['events'] = df['events'].apply(lambda x: 1 if x in hit_events else 0)
    print('events done')
    df['batter_righty'] = df['stand'].apply(lambda x: 1 if x == 'R' else 0)
    print('stand done')
    df['pitcher_righty'] = df['p_throws'].apply(lambda x: 1 if x == 'R' else 0)
    print('pitcher done')
    df['bottom'] = df['inning_topbot'].apply(lambda x: 1 if x == 'Bot' else 0)
    print('inning done')
    df = df.select_dtypes(exclude=['object'])
    print('dropping columns done')


    df.to_csv(str(year)+'_statcast_pbp.csv')
    return(df)
    

In [18]:
# probably best off grabbing all the data for each player and trimming it myself - speed up, cut down on api calls

    # instead of career stats we'll go off last two years to bias towards recency
def derived_cumulative_stats_v2(player, stats_type, start_date=None, end_date=datetime.strptime('2021-06-30', '%Y-%m-%d')):
    
    relevant_years = [str(end_date.year), str(end_date.year-1), str(end_date.year-2)]    
    
    history = pd.concat([
                pd.read_csv(relevant_years[0] + '_statcast_pbp.csv'),
                pd.read_csv(relevant_years[1] + '_statcast_pbp.csv'),
               pd.read_csv(relevant_years[2] + '_statcast_pbp.csv')
                ], ignore_index=True)

    if start_date == None:
        start_date = end_date-timedelta(weeks=52)
    if type(player) == str:
        player = player.split(' ')
        player.reverse()
        try:
            player_info = pybaseball.playerid_lookup(player[0], player[1])
            if len(player_info) >1:
                player_info = player_info.iloc[0]
                player = player_info['key_mlbam']
            elif player_info.empty:
                player_info = pybaseball.playerid_lookup(player[0], player[1], fuzzy=True).iloc[0]
                player = player_info['key_mlbam']
            else:
                try:
                    player = player_info.iloc[0]['key_mlbam']
                except:
                    raise NameError(player_info, player)
        except:
            raise NameError(player)
            
    history = history[history['game_type'] == 'R']
    history = history[(history['game_date'] >= start_date) & (history['game_date'] <= end_date)]

    if stats_type == 'pitcher':
        try:
            history = history[history['pitcher'] == player]
        except:
            raise NameError(player)

    #   PITCH METRICS
        pitches = {}
        pitches['pitch_hand'] = statsapi.player_stat_data(player)['pitch_hand']
        
    #     filter history down to PA-enders
        history = history[history['events'].isin([1, 0])]

    #    PLAYER METRICS
        games = history['game_pk'].unique()
        at_bat_list = []
        num_hits_list = []
        for game in games:
            game = history[history['game_pk'] == game]
            inning = game['inning'].max()
            at_bats = len(game)
            at_bat_list.append(at_bats)
            for i in range(1, inning+1):
                num_hits = len(game[(game['inning'] == i) & (game['events'] == 1)])
                num_hits_list.append(num_hits)
        pitches['games_played_last_2_years_pitcher'] = len(games)
        pitches['avg_PAs_per_apparence_pitcher'] = np.array(at_bat_list).mean()
        pitches['avg_hits_per_inning'] = np.array(num_hits_list).mean()

#         L/R splits
        left_pitcher = history[history['batter_righty'] == 0]
        right_pitcher = history[history['batter_righty'] == 1]
        l_pas = len(left_pitcher)
        r_pas = len(right_pitcher)
        l_hits = len(left_pitcher[(history['events'] == 1)])
        r_hits = len(right_pitcher[(history['events'] == 1)])

        if l_pas > 0 and r_pas > 0:
            pitches['H/PA_pitcher'] = (l_hits + r_hits)/(l_pas + r_pas)
            pitches['against_lefties_H/PA'] = (l_hits)/(l_pas)
            pitches['against_righties_H/PA'] = (r_hits)/(r_pas)
        elif l_pas > 0:
            pitches['H/PA_pitcher'] = (l_hits + r_hits)/(l_pas + r_pas)
            pitches['against_lefties_H/PA'] = (l_hits)/(l_pas)
            pitches['against_righties_H/PA'] = 0
        elif r_pas > 0:
            pitches['H/PA_pitcher'] = (l_hits + r_hits)/(l_pas + r_pas)
            pitches['against_lefties_H/PA'] = 0
            pitches['against_righties_H/PA'] = (r_hits)/(r_pas)
        else:
            pitches['H/PA_pitcher'] = 0
            pitches['against_lefties_H/PA'] = 0
            pitches['against_righties_H/PA'] = 0
        return(pitches)

    
    elif stats_type == 'batter':
        history = history[history['batter'] == player]


#     filter history down to PA-enders
        history = history[history['events'].isin([1, 0])]

#     H/PA per pitch type
#     K/PA per pitch type
#     H/PA vs righties, lefties
#     PA/G
#     avg_launch_angle
#     avg_launch_speed
#     xBA based on estimated_ba_using_speedangle



        # pitches = history['pitch_type'].unique()
        games = history['game_pk'].unique()
        pas_list = []

        batter = {}

        for game in games:
            pas = len(history[history['game_pk'] == game])
            # print(pas)
            pas_list.append(pas)
        batter['games_played_last_2_years_batter'] = len(games)
        batter['PA/G_batter'] = np.array(pas_list).mean()

#         righty/lefty split
        pitcher_right = history[history['pitcher_righty'] == 1]
        pas_r = len(pitcher_right)
        hits_r = len(pitcher_right[pitcher_right['events'] == 1])

        if pas_r > 0:
            batter['H/PA_against_R'] = hits_r/pas_r
        else:
            batter['H/PA_against_R'] = 0

        pitcher_left = history[history['pitcher_righty'] == 0]
        pas_l = len(pitcher_left)
        hits_l = len(pitcher_left[pitcher_left['events'] == 1])

        if pas_l > 0:
            batter['H/PA_against_L'] = hits_l/pas_l
        else:
            batter['H/PA_against_L'] = 0

        if pas_r > 0 or pas_l > 0:
            batter['H/PA_batter'] = (hits_r+hits_l)/(pas_r+pas_l)
        else:
            batter['H/PA_batter'] = 0
        batter['avg_launch_angle'] = history['launch_angle'].mean()
        batter['avg_launch_speed'] = history['launch_speed'].mean()
        batter['xBA'] = history['estimated_ba_using_speedangle'].mean()

        return(batter)

In [19]:
def prepreprocess_historic_data(df):

    pa_ending_events = np.array(['field_out',
                        'strikeout',
                        'single',
                        'walk',
                        'double',
                        'home_run',
                        'force_out',
                        'grounded_into_double_play',
                        'hit_by_pitch',
                        'field_error',
                        'sac_fly',
                        'triple',
                        'sac_bunt',
                        'fielders_choice',
                        'double_play',
                        'fielders_choice_out',
                        'strikeout_double_play',
                        'catcher_interf',
                        'sac_fly_double_play',
                        'triple_play',
                        'sac_bunt_double_play'])

    hit_events = np.array(['single',
                            'double',
                            'home_run',
                            'triple'])
    
    american_league_teams = np.array(['Boston Red Sox', 'Houston Astros', 'Chicago White Sox', 'Tampa Bay Rays', 'Oakland Athletics', 'Seattle Mariners', 'New York Yankees', 'Toronto Blue Jays', 'Los Angeles Angels', 'Cleveland Indians', 'Detroit Tigers', 'Kansas City Royals', 'Minnesota Twins', 'Texas Rangers', 'Baltimore Orioles'])
    
    df_filtered = df[df['events'].isin([1,0])]


    venue_coords = pd.read_csv('Parks.csv')
    
    game_pk_df = pd.DataFrame(columns = statsapi.schedule(game_id=566083)[0].keys())

    venue_dict = {}

    for index, row in df_filtered.iterrows():
        game_data = statsapi.schedule(game_id = row['game_pk'])[-1]
        venue = game_data['venue_name']
        game_pk_df = game_pk_df.append(game_data, ignore_index=True)

            
        try:
            city = venue_coords[venue_coords['NAME'] == venue]['CITY'].iloc[0]
            lat = venue_coords[venue_coords['NAME'] == venue]['Latitude'].iloc[0]
            lon = venue_coords[venue_coords['NAME'] == venue]['Longitude'].iloc[0]
            alt = venue_coords[venue_coords['NAME'] == venue]['Altitude'].iloc[0]
            venue_dict[venue] = np.array((city, lat, lon, alt))
        except IndexError:
            raise IndexError(venue, index)

    game_pk_df = game_pk_df.sort_values(['game_datetime'])

    

    game_pk_df['local_datetime'] = game_pk_df.apply(lambda row: convert_UTC_to_local(row), axis=1)
    game_pk_df['local_datetime'] = game_pk_df['local_datetime'].apply(lambda x: x.strftime("%Y-%m-%dT%H:%M:%S"))
    
    
    datetime_coordinate_matching = {}
    for index, row in game_pk_df.iterrows():
        city, lat, lon, alt = venue_dict[row['venue_name']]
        dc_datetime = row['local_datetime']
        if dc_datetime in datetime_coordinate_matching.keys():
            datetime_coordinate_matching[dc_datetime].append(','.join([str(lat), str(lon)]))
        else:
            datetime_coordinate_matching[dc_datetime] = [','.join([str(lat), str(lon)])]

    game_pk_df['coordinates'] = game_pk_df['venue_name'].apply(lambda x: ','.join(venue_dict[x][1:3]))
    game_pk_df['alt'] = game_pk_df['venue_name'].apply(lambda x: venue_dict[x][3])

    from api_keys import visual_crossing
    import requests

    weather_df = pd.DataFrame()
    for key, value in datetime_coordinate_matching.items():
        url_locations = '|'.join(value)
        URL = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?&aggregateHours=1&startDateTime={key}&endDateTime={key}&unitGroup=us&contentType=json&location={url_locations}&key={visual_crossing}'
        global api_counter
        if api_counter == 100000000:
            print('Prepreprocessing will continue at ' + (datetime.now()+timedelta(seconds = 86400)).strftime('%H:%M:%S'))
            sleep(86400)
            api_counter = 0
        try:
            api_counter += 1
            response = requests.get(URL)
            data = response.json()
            locations = list(data['locations'].keys())
            for each in locations:
                values = data['locations'][each]['values'][0]
                weather_df = weather_df.append({**{'coordinates': each}, **values}, ignore_index=True)
        except:
            raise IndexError(key, value, response)

    weather_df['datetimeStr'] = weather_df['datetimeStr'].apply(lambda x: x[:-6])
    weather_df = weather_df.rename(columns={'datetimeStr': 'local_datetime'})

    games_and_weather = pd.merge(
        game_pk_df,
        weather_df,
        how="left",
        on=None,
        left_on=['local_datetime', 'coordinates'],
        right_on=['local_datetime', 'coordinates'],
        left_index=False,
        right_index=False,
        sort=False,
        suffixes=("_gpk", "_acw"),
        copy=True,
        indicator=False,
        validate=None,
    )

    df_detailed = pd.merge(
        games_and_weather,
        df_filtered,
        how="right",
        on=None,
        left_on='game_id',
        right_on='game_pk',
        left_index=False,
        right_index=False,
        sort=False,
        suffixes=("_gaw", "_dff"),
        copy=True,
        indicator=False,
        validate=None,
    )

    print(df_detailed.info())


    df_detailed['covid_doubleheader'] = df_detailed.apply(lambda row: 1 if row['game_year'] in [2020, 2021] and row['doubleheader'] =='Y' else 0, axis=1)

    

    df_detailed['designated_hitter'] = df_detailed.apply(lambda row: 1 if row['home_name'] in american_league_teams or row['game_year'] == 2020 else 0, axis = 1)

    



    pitcher_df = pd.concat([df_detailed[['game_id', 'game_date_gaw', 'away_probable_pitcher']].rename(columns={'away_probable_pitcher': 'probable_pitcher'}), df_detailed[['game_id', 'game_date_gaw', 'home_probable_pitcher']].rename(columns={'home_probable_pitcher': 'probable_pitcher'})])
    pitcher_df = pitcher_df.drop_duplicates()
    pitcher_df['probable_pitcher'] = pitcher_df['probable_pitcher'].replace('', np.nan)
    pitcher_df = pitcher_df.dropna(subset=['probable_pitcher'])
    pitcher_df = pitcher_df.reset_index(drop=True)

    pitcher_stats = pitcher_df.apply(lambda row: pd.Series(derived_cumulative_stats_v2(row['probable_pitcher'], 'pitcher', end_date=row['game_date_gaw']-timedelta(days=1))), axis = 1)

    pitcher_df = pd.merge(
        pitcher_stats,
        pitcher_df,
        how="left",
        on=None,
        left_on=None,
        right_on=None,
        left_index=True,
        right_index=True,
        sort=False,
        suffixes=("_s", "_df"),
        copy=True,
        indicator=False,
        validate=None,
    )

    home_batters = df_detailed[df_detailed['bottom'] == 1]
    away_batters = df_detailed[df_detailed['bottom'] == 0]

    df_wpitching_p1 = pd.merge(
        pitcher_df,
        home_batters,
        how="right",
        on=None,
        left_on=['game_id', 'probable_pitcher'],
        right_on=['game_id', 'away_probable_pitcher'],
        left_index=False,
        right_index=False,
        sort=False,
        suffixes=("_p", "_df"),
        copy=True,
        indicator=False,
        validate=None,
    )

    df_wpitching_p2 = pd.merge(
        pitcher_df,
        away_batters,
        how="right",
        on=None,
        left_on=['game_id', 'probable_pitcher'],
        right_on=['game_id', 'home_probable_pitcher'],
        left_index=False,
        right_index=False,
        sort=False,
        suffixes=("_p", "_df"),
        copy=True,
        indicator=False,
        validate=None,
    )
    df_wpitching = pd.concat([df_wpitching_p1, df_wpitching_p2], ignore_index=True)
    df_wpitching = df_wpitching.sort_values(['game_date_gaw_df', 'game_pk','home_team', 'inning', 'bottom', 'outs_when_up', 'at_bat_number', 'pitch_number']).reset_index(drop=True)

    df_wpitching['got_a_hit'] = df_wpitching.apply(lambda row: 1 if row['events'] == 1 else 0, axis=1)
    df_wpitching = df_wpitching.sort_values('got_a_hit').drop_duplicates(subset=['game_id', 'batter'], keep='last').sort_values(['game_date_gaw_df', 'game_pk','home_team', 'inning', 'bottom', 'outs_when_up', 'at_bat_number', 'pitch_number']).reset_index(drop=True)

    batting_stats = df_wpitching.apply(lambda row: pd.Series(derived_cumulative_stats_v2(row['batter'],'batter', end_date=row['game_date_gaw']-timedelta(days=1))), axis = 1)

    df_final = pd.merge(
        batting_stats,
        df_wpitching,
        how="right",
        on=None,
        left_on=None,
        right_on=None,
        left_index=True,
        right_index=True,
        sort=False,
        suffixes=("_bs", "_df"),
        copy=True,
        indicator=False,
        validate=None,
    )

    df_final = df_final.rename(columns={'games_played_last_2_years_df': 'games_played_last_2_years_pitcher', 'games_played_last_2_years_bs': 'games_played_last_2_years_batter', 'bottom': 'home'})

    df_final['local_datetime'] = pd.to_datetime(df_final['local_datetime'])
    df_final['local_date'] = df_final['local_datetime'].apply(lambda x: x.toordinal())
    df_final['local_datetime'] = df_final['local_datetime'].apply(lambda x: x.timestamp())


    df_final =df_final.drop(columns= ['player_name',
     'game_type_dff', 'pitch_type',
     'release_speed',
     'release_pos_x',
     'release_pos_z',
     'pitcher',
     'events',
     'description',
     'spin_dir',
    'spin_rate_deprecated',
     'break_angle_deprecated',
     'break_length_deprecated',
     'zone',
     'des',
      'p_throws',
    'type',
     'hit_location',
     'bb_type',
     'balls',
     'strikes',
     'pfx_x',
     'pfx_z',
     'plate_x',
     'plate_z',
     'on_3b',
     'on_2b',
     'on_1b',
     'outs_when_up',
     'inning',
     'hc_x',
     'hc_y',
     'tfs_deprecated',
     'tfs_zulu_deprecated',
      'sv_id',
     'vx0',
     'vy0',
     'vz0',
     'ax',
     'ay',
     'az',
     'sz_top',
     'sz_bot',
     'hit_distance_sc',
     'launch_speed',
     'launch_angle',
     'effective_speed',
     'release_spin_rate',
     'release_extension',
     'pitcher.1',
     'fielder_2.1',
     'fielder_3',
     'fielder_4',
     'fielder_5',
     'fielder_6',
     'fielder_7',
     'fielder_8',
     'fielder_9',
     'release_pos_y',
     'estimated_ba_using_speedangle',
     'estimated_woba_using_speedangle',
     'woba_value',
     'woba_denom',
     'babip_value',
     'iso_value',
     'launch_speed_angle',
     'at_bat_number',
     'pitch_number',
     'pitch_name',
     'home_score_dff',
     'away_score_dff',
     'bat_score',
     'fld_score',
     'post_away_score',
     'post_home_score',
     'post_bat_score',
     'post_fld_score',
     'if_fielding_alignment',
     'of_fielding_alignment',
     'spin_axis',
     'delta_home_win_exp',
     'delta_run_exp',
     'game_pk',
     'game_id','game_date_gaw_p',
     'game_datetime',
     'game_date_gaw_df',
     'game_type_gaw',
     'datetime',
    #  'datetimeStr',
     'game_date_dff','winning_team',
     'losing_team',
     'winning_pitcher',
     'losing_pitcher',
     'save_pitcher',
     'summary', 'home_probable_pitcher',
     'away_probable_pitcher',
     'home_pitcher_note',
     'away_pitcher_note',
     'away_score_gaw',
     'home_score_gaw',
     'current_inning',
     'inning_state',
     'venue_id',
     'status',
     'home_team', 'away_team', 'home_id', 'away_id'])
    
    for col in df_final.columns.to_list():
        if len(df_final[col].unique()) == 1:
            df_final = df_final.drop([col], axis=1)

In [ ]:
for year in [2017, 2018, 2019, 2020, 2021]:
    statcast_importer(year)

In [ ]:
start = datetime.now()
sample = prepreprocess_historic_data(pd.concat([
                pd.read_csv('2019_statcast_pbp.csv'),
                pd.read_csv('2020_statcast_pbp.csv'),
               pd.read_csv('2021_statcast_pbp.csv')
                ], ignore_index=True))
print(datetime.now() - start)
start = datetime.now()
holdout = prepreprocess_historic_data(pybaseball.statcast('2021-07-01', (datetime.now()-timedelta(weeks=1)).strftime('%Y-%m-%d')))
print(datetime.now() - start)

In [ ]:
def lookup_mlb():
    """Get list of MLB team IDs for the purposes of pulling active rosters"""
    params = {
        "activeStatus": 'Y',
        "season": 2021,
        "sportIds": 1,
        "fields": "teams,id,name,league",
    }
    
    r= statsapi.get('teams', params)
    
    teams = {}
    for team in r['teams']:
        if team['league']['id'] in [103, 104]:
            teams[team['name']] = team['id']
    
    return (teams)

def roster(teamId, rosterType=None, season=datetime.now().year, date=None):
    """Get the roster for a given team."""
    if not rosterType:
        rosterType = "active"

    params = {"rosterType": rosterType, "season": season, "teamId": teamId}
    if date:
        params.update({"date": date})

    r = statsapi.get("team_roster", params)

    players = []
    for x in r["roster"]:
        players.append(
            x["person"]["id"]
        )

    
    return (players)

def lookup_team(lookup_value, activeStatus="Y", season=datetime.now().year, sportIds=1):
    """Get a info about a team or teams based on the team name, city, abbreviation, or file code."""
    params = {
        "activeStatus": activeStatus,
        "season": season,
        "sportIds": sportIds,
        "fields": "teams,id,name,teamCode,fileCode,teamName,locationName,shortName,league",
    }
    r = statsapi.get("teams", params)

    teams = []
    for team in r["teams"]:
        for v in team.values():
            if str(lookup_value).lower() in str(v).lower():
                teams.append(team)
                break

    return teams

# MODEL WORK

In [ ]:
def evaluate(estimator, X_train, X_val, y_train, y_val, holdout, roc_auc='proba', output = False):

    X_holdout = holdout.drop(['got_a_hit'], axis=1)
    y_holdout = holdout['got_a_hit']
    
    #     grab predictions
    train_preds = estimator.predict(X_train)
    val_preds = estimator.predict(X_val)
    holdout_preds = estimator.predict(X_holdout)
    
#     output needed for roc_auc score
    if roc_auc == 'skip':
        train_out = False
        val_out = False
        holdout_out = False
    elif roc_auc == 'dec': # not all classifiers have a decision function
        train_out = estimator.decision_function(X_train)
        val_out = estimator.decision_function(X_val)
        holdout_out = estimator.decision_function(X_holdout)
    elif roc_auc == 'proba':
        try:
            train_out = estimator.predict_proba(X_train)[:, 1]
            val_out = estimator.predict_proba(X_val)[:, 1]
            holdout_out = estimator.predict_proba(X_holdout)[:, 1]
        except AttributeError:
            train_out = estimator.predict(X_train)
            val_out = estimator.predict(X_val)
            holdout_out = estimator.predict(X_holdout)

    else:
        raise Exception("The value for roc_auc should be 'skip', 'dec', or 'proba'.")
    
    ac = accuracy_score(y_train, train_preds)
    f1 = f1_score(y_train, train_preds)
    ras = roc_auc_score(y_train, train_out)
    pr = precision_score(y_train, train_preds)
    
    if output == True:
        print('Train Scores')
        print('------------')
        print(f'Accuracy: {ac}')
        print(f'F1 Score: {f1}')
        if type(train_out) == np.ndarray:
            print(f'ROC-AUC: {ras}')
        print(f'Precision: {pr}')

    ac = accuracy_score(y_val, val_preds)
    f1 = f1_score(y_val, val_preds)
#     print(type(y_val))
    ras = roc_auc_score(y_val, val_out)
    pr = precision_score(y_val, val_preds)
    
    if output == True:
        print('-----------------------------------')
        print('Val Scores')
        print('-----------')
        print(f'Accuracy: {ac}')
        print(f'F1 Score: {f1}')
        if type(val_out) == np.ndarray:
            print(f'ROC-AUC: {ras}')
        print(f'Precision: {pr}')
    
    ac = accuracy_score(y_holdout, holdout_preds)
    f1 = f1_score(y_holdout, holdout_preds)
#     print(type(holdout['got_a_hit']))
    ras = roc_auc_score(y_holdout, holdout_out)
    pr = precision_score(y_holdout, holdout_preds)
    
    if output == True:
        print('-----------------------------------')
        print('Holdout Scores')
        print('-----------')
        print(f'Accuracy: {ac}')
        print(f'F1 Score: {f1}')
        if type(holdout_out) == np.ndarray:
            print(f'ROC-AUC: {ras}')
        print(f'Precision: {pr}')

        print('\nVal Data')
        print('-----------')

        plot_confusion_matrix(estimator, X_val, y_val, values_format=',.5g')
        plt.show()
        
        print('Holdout Data')        
        print('-----------')

        plot_confusion_matrix(estimator, X_holdout, y_holdout, values_format=',.5g')
        plt.show()

In [ ]:
def streak_checker(dataframe, output = False, metric = 'odds', ascending=False, combiner=None, timed=False):
    results = []

    try:
        if 'L/R_split' in metric and combiner == 'add':
            if timed == True:
                start = datetime.now()

                print(combiner, start)
            dataframe_copy = dataframe.copy()
            dataframe_copy['temp'] = dataframe.apply(lambda row: row['H/PA_against_R'] + row['against_lefties_H/PA'] if (row['pitch_hand'] == 'Right' and row['stand'] == 'L') else (row['H/PA_against_L'] + row['against_righties_H/PA'] if (row['pitch_hand'] == 'Left' and row['stand'] == 'R') else 0), axis=1)
            mms = MinMaxScaler()
            dataframe_copy['temp'] = pd.DataFrame(mms.fit_transform(dataframe_copy[['temp']]))
            
            if timed == True:
                print(datetime.now() - start)
            dataframe = dataframe_copy
            metric='temp'
        elif 'L/R_split' in metric and combiner == 'multiply':
            if timed == True:
                start = datetime.now()

                print(combiner, start)
            dataframe_copy = dataframe.copy()
            dataframe_copy['temp'] = dataframe.apply(lambda row: row['H/PA_against_R'] * row['against_lefties_H/PA'] if (row['pitch_hand'] == 'Right' and row['stand'] == 'L') else (row['H/PA_against_L'] * row['against_righties_H/PA'] if (row['pitch_hand'] == 'Left' and row['stand'] == 'R') else 0), axis=1)
            mms = MinMaxScaler()
            dataframe_copy['temp'] = pd.DataFrame(mms.fit_transform(dataframe_copy[['temp']]))
            
            if timed == True:
                print(datetime.now() - start)
            dataframe = dataframe_copy
            metric='temp'
        if type(metric) == list:
            dataframe_copy = dataframe.copy()
            if combiner == 'add':
                if timed == True:
                    start = datetime.now()

                    print(combiner, start)
                dataframe_copy['temp'] = dataframe.apply(lambda row: row[metric[0]] + row[metric[1]], axis=1)
                mms = MinMaxScaler()
                dataframe_copy['temp'] = pd.DataFrame(mms.fit_transform(dataframe_copy[['temp']]))
                
                if timed == True:
                    print(datetime.now() - start)
            elif combiner == 'multiply':
                if timed == True:
                    start = datetime.now()

                    print(combiner, start)
                dataframe_copy['temp'] = dataframe.apply(lambda row: row[metric[0]] * row[metric[1]], axis=1)
                mms = MinMaxScaler()
                dataframe_copy['temp'] = pd.DataFrame(mms.fit_transform(dataframe_copy[['temp']]))
                
                if timed == True:
                    print(datetime.now() - start)
            
            else:
                raise KeyError(f'{combiner} combiners have not been implemented yet')
            dataframe = dataframe_copy
            metric = 'temp'
        else:
            pass
    except KeyError:
        raise KeyError("metric must be chosen from dataframe's columns")
    
    date_list = np.sort(dataframe['local_date'].unique())
    
    for day in date_list:
        best_bet = dataframe[dataframe['local_date'] == day].sort_values(metric, ascending=ascending).iloc[0]
        
        odds = best_bet[metric]
        date = datetime.fromordinal(best_bet['local_date']).strftime("%b %d %Y")
        
        player_info = pybaseball.playerid_reverse_lookup([best_bet['batter']])
        if len(player_info) >1:
            player_info = player_info.iloc[0]
            player = player_info['key_mlbam']
        elif player_info.empty:
            player_info = playerid_lookup(player[0], player[1], fuzzy=True).iloc[0]
            player = player_info['key_mlbam']
        else:
            try:
                player = ' '.join(player_info.iloc[0][['name_first', 'name_last']])
            except:
                raise NameError(player_info, player)
        if best_bet['home'] == 0:
            team = best_bet['away_name']
        else:
            team = best_bet['home_name']
    #     team = statsapi.lookup_team(player['currentTeam']['id'])[0]['name']
    #     player_name = player['fullName']
        actual_result = best_bet['got_a_hit']
        results.append(actual_result)
        if output == True:
            if day > date_list[-20]:
                pass
#                 print(date, player, odds, actual_result)
    longest = 0
    current = 0
    for num in results:
        if num == 1:
            current += 1
        else:
            longest = max(longest, current)
            current = 0
    if output == True:
        print(f'Longest streak in set ({len(results)} days): ', longest)
        print(f'Total correct guesses in set ({len(results)} days): ', sum(results), sum(results)/len(results))
#     return (dataframe, best_bet)

In [ ]:
def run_model(clf, X_train, X_val, y_train, y_val, holdout):
    probability = True
    
    clf.fit(X_train.head(100), y_train.head(100))

    evaluate(clf, X_train.head(100), X_val.head(100), y_train.head(100), y_val.head(100), holdout);

    runtime = datetime.now() - start

    if runtime > timedelta(seconds=5):
        print(f'Model would take {runtime.total_seconds()*200/60} minutes to run.')
    else:
        print('Model should take between {:.0f} and {:.0f} minutes to run and finish by {}'.format(runtime.total_seconds()*20/60, runtime.total_seconds()*200/60, (datetime.now()+(timedelta(seconds=(runtime.total_seconds()*200)))).strftime("%H:%M")))
        clf.fit(X_train, y_train)
        evaluate(clf, X_train, X_val, y_train, y_val, holdout, output=True)
        plot_roc_curve(clf, X_val, y_val)
        plt.show()
        
        try:
            val_df_odds = pd.Series(clf.predict_proba(X_val)[:, 1], name='odds')
            holdout_df_odds = pd.Series(clf.predict_proba(holdout.drop(['got_a_hit'], axis=1))[:, 1], name='odds')
        except AttributeError:
            probability = False
            val_df_odds = pd.Series(clf.predict(X_val), name= 'odds')
            holdout_df_odds = pd.Series(clf.predict(holdout.drop(['got_a_hit'], axis=1)), name='odds')

        val_df = X_val.assign(got_a_hit = y_val).reset_index(drop=True)
        val_df = val_df.assign(odds=val_df_odds)
        holdout = holdout.reset_index(drop=True)
        holdout = holdout.assign(odds=val_df_odds)
        if set(val_df_odds.value_counts().index) == set([1, 0]):
            print(str(clf.get_params()['steps'][1][1]), "doesn't return probabilities, so no streak results will be returned based on this alone.")
            pass
        else:
            print('-----------------------------------')

            print('Val data')
            print('-----------')

            streak_checker(val_df, output=True)
            print('Holdout data')
            print('-----------')

            streak_checker(holdout, output=True)
            
        print('Odds multiplied by PA/G_batter')
        print('-----------')
        
        print('Val data')
        print('-----------')
        streak_checker(val_df, output=True, metric=['PA/G_batter', 'odds'], combiner='multiply')
        print('Holdout data')
        print('-----------')
        streak_checker(holdout, output=True, metric=['PA/G_batter', 'odds'], combiner='multiply')
        
        print('Odds added to PA/G_batter')
        print('-----------')

        print('Val data')
        print('-----------')
        streak_checker(val_df, output=True, metric=['L/R_split', 'odds'], combiner='add')
        print('Holdout data')
        print('-----------')
        streak_checker(holdout, output=True, metric=['L/R_split', 'odds'], combiner='add')

In [ ]:
def train_test_split(df, test_size=.25):
    
    '''My simplified train_test_split which just cuts the dataframe by date, in order to accurately evaluate streak results, to roughly the test size requested'''
    
    sample_size = len(X)
    
    train_size = 1 - test_size
    
    train_size = round(train_size * sample_size)
    
    df = df.sort_values(['local_date']).reset_index(drop=True)
    
    date_cutoff = df.iloc[train_size]['local_date']

    train = df[df['local_date'] <= date_cutoff]
    val = df[df['local_date'] > date_cutoff]
    
    X_train = train.drop(['got_a_hit'], axis=1)
    y_train = train['got_a_hit']
    
    X_val = val.drop(['got_a_hit'], axis=1)
    y_val = val['got_a_hit']
    
    return(X_train, X_val, y_train, y_val)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df)

In [ ]:
num_cols = [] # all numeric columns
cols_to_ohe = [] # doubleheader, conditions, stand
cols_to_targ = [] # all other object columns

for c in X_train.columns:
    if X_train[c].dtype in ['float64', 'int64'] and c not in ['game_num', 'batter', 'fielder_2', 'umpire']:
        num_cols.append(c)
    elif len(X_train[c].unique()) < 10:
        cols_to_ohe.append(c)
    else:
        cols_to_targ.append(c)

In [ ]:
num_transformer = Pipeline(steps=[
    ('num_imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

ohe_transformer = Pipeline(steps=[
    ('ohe_imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('ohencoder', OneHotEncoder(handle_unknown='ignore'))
])

target_encoder = Pipeline(steps=[
    ('freq_enc', ce.target_encoder.TargetEncoder()),
    ('freq_imputer', SimpleImputer(strategy='constant', fill_value=0))
    
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),
        ('ohe', ohe_transformer, cols_to_ohe),
        ('target', target_encoder, cols_to_targ)
    ])

# Baseline - dummy models
Pick player with the highest Plate Appearances per Game over the last 2 years

In [ ]:
streak_checker(df, output=True, metric='PA/G_batter')
streak_checker(test_df, output=True, metric='PA/G_batter')

Observations - this seems to trend towards a small group of players except for when a player makes their mLB debut towards the top of the lineup (Wander Franco, Greg Deichmann). Sample size is an issue - could be addressed by either looking at games played or excluding PA/G above 4.9.



Pick player with highest launch speed

In [ ]:
streak_checker(df, output=True, metric='avg_launch_speed')
streak_checker(test_df, output=True, metric='avg_launch_speed')

The metrics point to this being mostly useless which is backed up by the fact that it seems to occasionally choose pitchers (Hunter Strickland, Zack Greinke), when it's not fawning over Aaron Judge. Could be worth excluding pitchers as a class though would be hard to implement with the current dataset, not to mention the fact that 2-way players would need to be excluded from that filter.

In [ ]:
streak_checker(df, output=True, metric=['avg_launch_angle', 'avg_launch_speed'], combiner='add')
streak_checker(test_df, output=True, metric=['avg_launch_angle', 'avg_launch_speed'], combiner='add')

Exaserbates bias towards small sample sizes, leading part time players to dominate selection rather than proven hitters

Would also be worth checking lefty righty splits

In [ ]:
streak_checker(df, output=True, metric='L/R_split', combiner='add')
streak_checker(test_df, output=True, metric='L/R_split', combiner='add')
streak_checker(df, output=True, metric='L/R_split', combiner='multiply')
streak_checker(test_df, output=True, metric='L/R_split', combiner='multiply')

Nothing stands out in these groups. Seems to be a mix of journeymen and all-stars.

# Model 1 - Decision Tree

In [ ]:
start = datetime.now()
dt = DecisionTreeClassifier()

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('tree', dt)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 2: Logistic Regression

In [ ]:
start = datetime.now()

logreg = LogisticRegression(solver='sag')

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('logreg', logreg)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 3: SVC too slow, linearSVC doesn't supply probability, so I need to supply a secondary way to select a single batter per day

In [ ]:
start = datetime.now()

Linearsvc = LinearSVC()


clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('linearsvc', Linearsvc)
])


run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 4: KNN

In [ ]:
start = datetime.now()

knn = KNeighborsClassifier(n_jobs=-1)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('knn', knn)
])


run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 5: Random Forest

In [ ]:
start = datetime.now()

rfc = RandomForestClassifier(n_jobs=-1)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('rfc', rfc)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 6: AdaBoost


In [ ]:
start = datetime.now()

ada = AdaBoostClassifier(random_state=42)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('ada', ada)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 7: GradientBoost


In [ ]:
start = datetime.now()

gbm = GradientBoostingClassifier()
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('gbm', gbm)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 8: XGBoost


In [ ]:
start = datetime.now()

xbg = xgb.XGBClassifier(random_state=42)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('xbg', xbg)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 9: Random Forest with Grid Search


In [ ]:
start = datetime.now()

parameters = {
    'min_samples_split': [3, 5, 100], 
    'n_estimators' : [100, 300],
        'max_depth': [3, 5, 15, 25],

}

run_model(clf, X_train, X_val, y_train, y_val, test_df)

In [ ]:
# to be run daily
def daily_guess():

    # get game data, then weather data, then pitcher data, then batter data

    todays_games = pd.DataFrame()


    for game in statsapi.schedule():
    #     grab 'away_probable_pitcher','home_probable_pitcher','away_name',
    #     'home_name','doubleheader','game_num','venue_name','game_datetime'

    #     df['local_datetime'] = pd.to_datetime(df['local_datetime'])
    #     df['local_date'] = df['local_datetime'].apply(lambda x: x.toordinal())
    #     df['local_datetime'] = df['local_datetime'].apply(lambda x: x.timestamp())

        row = {
            'game_id': game['game_id'],
            'away_probable_pitcher': game['away_probable_pitcher'],
            'home_probable_pitcher': game['home_probable_pitcher'],
            'away_name': game['away_name'],
            'home_name': game['home_name'],
            'doubleheader': game['doubleheader'],
            'game_num': game['game_num'],
            'venue_name': game['venue_name'],
            'game_datetime': game['game_datetime']
        }

        todays_games = todays_games.append(row, ignore_index=True)

    todays_games = todays_games.replace('', np.nan)    

    games_and_weather = time_date_weather_data(todays_games)

    away_pitchers = games_and_weather['away_probable_pitcher']
    home_pitchers = games_and_weather['home_probable_pitcher']
    pitchers_stats = pd.DataFrame()

    for a in away_pitchers:
        print(a)
        if type(a) == float:
            pitchers_stats = pitchers_stats.append(pd.Series(), ignore_index=True)
        else:
            pitchers_stats = pitchers_stats.append({**derived_cumulative_stats_v2(a, 'pitcher', end_date=(datetime.now()-timedelta(seconds=86400)).strftime("%Y-%m-%d")), **{'probable_pitcher': a}}, ignore_index=True)

    for h in home_pitchers:
        print(h)
        if type(h) == float:
            pitchers_stats = pitchers_stats.append(pd.Series(), ignore_index=True)
        else:
            pitchers_stats = pitchers_stats.append({**derived_cumulative_stats_v2(h, 'pitcher', end_date=(datetime.now()-timedelta(seconds=86400)).strftime("%Y-%m-%d")), **{'probable_pitcher': h}}, ignore_index=True)

    batting_stats = pd.DataFrame()
    for index, game in games_and_weather.iterrows():
        print(game[['away_name', 'home_name']])
    #     home batters, away pitcher
        batting_team_id = lookup_mlb()[game['home_name']]
        batting_roster = roster(batting_team_id)
        for b in batting_roster:
            start = datetime.now()
            batting_stats = batting_stats.append({**derived_cumulative_stats_v2(b, 'batter', end_date=(datetime.now()-timedelta(seconds=86400)).strftime("%Y-%m-%d")), **pitchers_stats[pitchers_stats['probable_pitcher'] == game['away_probable_pitcher']], **game[['away_name',
                                             'home_name',
                                             'doubleheader',
                                             'game_num',
                                             'venue_name',
                                             'local_datetime',
                                             'coordinates',
                                             'cloudcover',
                                             'conditions',
                                             'dew',
                                             'heatindex',
                                             'humidity',
                                             'maxt',
                                             'mint',
                                             'precip',
                                             'sealevelpressure',
                                             'snowdepth',
                                             'temp',
                                             'visibility',
                                             'wdir',
                                             'weathertype',
                                             'wgust',
                                             'windchill',
                                             'wspd'
     ]]}, ignore_index=True)
            print(datetime.now()-start)
        #     away batters, home pitcher
        batting_team_id = lookup_mlb()[game['away_name']]
        batting_roster = roster(batting_team_id)
        for b in batting_roster:
            batting_stats = batting_stats.append({**derived_cumulative_stats_v2(b, 'batter', end_date=(datetime.now()-timedelta(seconds=86400)).strftime("%Y-%m-%d")), **pitchers_stats[pitchers_stats['probable_pitcher'] == game['home_probable_pitcher']], **game[['away_name',
                                             'home_name',
                                             'doubleheader',
                                             'game_num',
                                             'venue_name',
                                             'local_datetime',
                                             'coordinates',
                                             'cloudcover',
                                             'conditions',
                                             'dew',
                                             'heatindex',
                                             'humidity',
                                             'maxt',
                                             'mint',
                                             'precip',
                                             'sealevelpressure',
                                             'snowdepth',
                                             'temp',
                                             'visibility',
                                             'wdir',
                                             'weathertype',
                                             'wgust',
                                             'windchill',
                                             'wspd'
     ]]}, ignore_index=True)

# TODO
# RUN batting_stats through predictors